Import libraries

In [1]:
import numpy as np
import cvxpy as cp

Define the given initial parameters like cost, number of units of each currency, etc

In [2]:

tickers = ["USD", "EUR", "GBP", "CAD", "JPY", "CNY", "RUB", "MXN", "INR", "BRL"]
n = len(tickers)#no of types of currencies
F = np.zeros((n, n))#matrix to compute exchange rates
# USD
data = ([1.0, 0.87, 0.76, 1.31, 108.90, 6.72, 65.45, 19.11, 71.13, 3.69],
# EUR
[1.0, 0.88, 1.51, 125.15, 7.72, 75.23, 21.96, 81.85, 4.24],
# GBP
[1.0, 1.72, 142.94, 8.82, 85.90, 25.08, 93.50, 4.84],
# CAD
[1.0, 82.93, 5.11, 49.82, 14.54, 54.23, 2.81],
# JPY
[1.0, 0.062, 0.60, 0.18, 0.65, 0.034],
# CNY
[1.0, 9.74, 2.85, 10.61, 0.55],
# RUB
[1.0, 0.29, 1.09, 0.056],
# MXN
[1.0, 3.73, 0.19],
# INR
[1.0, 0.052],
# BRL
[1.0])
for i in range(n):
    F[i,i:] = data[i]
for j in range(n):
    for i in range(j+1,n):
        F[i,j] = 1.035/F[j,i]
print("Shape of F matrix",F.shape)

        
# Initial and final portfolios.
c_req = np.arange(1,n+1)
c_req = 1e4*c_req/c_req.sum()
c_init = c_req[::-1]#initial counts in hand


Shape of F matrix (10, 10)


Define the variable with exchanges made

In [3]:
X=cp.Variable((n,n))

If c_init is the initial cost, X units are exchanged, and X/F units are obtained back. Then c_final is the currency in hand finally post exchange
If conversion rate is known(mentioned as to USD in qn), then cost of a exchange is computed as difference of initial and final in hand currency multiplied by conversion rate 

In [4]:
c_final=c_init+(X/F)@np.ones(n)-X.T@np.ones(n)
conversion_rate=np.sqrt(F[:,0]/F[0,:])
cost=(c_init-c_final)@conversion_rate

Define objective as to minimize the cost involved, constraints expressed in qn and finally the problem using the above two.

In [5]:
objective=cp.Minimize(cost)
constraints=[X>=0,c_final>=c_req,c_init>=X.T@np.ones(n),cp.diag(X)==0]
problem=cp.Problem(objective,constraints)


Compute the cost by solving the problem

In [6]:
print("Minimum cost :", problem.solve())

Minimum cost : 7.720059340058015


Compute the optimal value of X that resulted in minimum cost obtained.

In [9]:
print("Exchange made are : ", X.value)

Exchange made are :  [[6.23500805e-14 4.26378962e-09 3.69571711e-09 5.64615472e-09
  3.28710650e-07 1.61321086e-08 1.52124644e-07 4.45680500e-08
  1.28905492e-07 5.88965811e-09]
 [5.05940761e-09 6.23513554e-14 8.66968379e-09 2.18217089e-08
  1.25513932e-06 3.31083355e-08 3.09318779e-07 8.83503561e-08
  1.95695253e-07 8.91695364e-09]
 [5.06144792e-09 2.78991627e-09 6.23537860e-14 5.45454545e+02
  2.19926086e-06 3.75451490e-08 3.67347691e-07 1.01892604e-07
  2.11540956e-07 9.74007895e-09]
 [5.55175378e-09 2.51244215e-09 1.88832907e-09 6.23477978e-14
  6.78131186e-06 4.93100738e-08 4.50037900e-07 1.25444863e-07
  2.37110316e-07 1.01946294e-08]
 [1.01153910e-08 2.71050628e-09 2.17947362e-09 3.48820899e-09
  6.23430103e-14 4.06564748e-08 8.83995727e-07 4.38317197e-08
  9.49168016e-07 1.01810185e-08]
 [1.65029401e+01 4.03105383e-09 3.17630883e-09 4.65764454e-09
  4.05623583e-07 6.23420970e-14 7.27272724e+02 6.53362004e-07
  3.74312655e-06 1.61201962e-08]
 [1.17316464e-08 3.04283792e-09 2.401